In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import uuid, json, re


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD_Python_Module import AnimalShelter
UID = uuid.uuid4().hex[:8]

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

AAC_USER = "aacuser"
AAC_PASS = "ILoveAnimals92!"

# Connect to database via CRUD Module
shelter = AnimalShelter(USER=AAC_USER, PASS=AAC_PASS)

def fetch_df(query: dict):
    rows = shelter.read(query, projection=None, limit=0)
    df = pd.DataFrame(rows).fillna("")
    return df

# initial unfiltered load
df = fetch_df({})

if "_id" in df.columns:
    df = df.drop(columns=["_id"])

# print(len(df))
# print(df.columns.tolist())


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# from base64 import b64encode
# from pathlib import Path

# # robust path resolution from the notebook's working dir
# LOGO_PATHS = [
#     Path("code_files") / "Grazioso Salvare Logo.png", 
# ]

# LOGO_SRC = None
# for p in LOGO_PATHS:
#     if p.exists():
#         LOGO_SRC = "data:image/png;base64," + b64encode(p.read_bytes()).decode("utf-8")
#         break

# if LOGO_SRC is None:
#     raise FileNotFoundError("Logo not found.")

datatable_id = f"datatable-{UID}"
map_id       = f"map-{UID}"
pie_id       = f"pie-{UID}"
filter_id    = f"filters-{UID}"
uid_label_id = f"uid-{UID}"

app.layout = html.Div([
    html.Div([
        html.A(
            html.Img(src="/code_files/Grazioso Salvare Logo.png", style={"height":"56px"}),
            href="https://www.snhu.edu", target="_blank"
        ),
        html.Span(f"  |  Dashboard by Shauntel Hall  |  UID: {UID}",
                  style={"marginLeft":"10px","color":"#555"})
    ], style={"display":"flex","alignItems":"center","gap":"10px"}),
    html.Hr(),

    # --- Filters (view/controller inputs) ---
    dcc.RadioItems(
        id=filter_id,
        options=[
            {"label":"Water Rescue","value":"water"},
            {"label":"Mountain / Wilderness Rescue","value":"mountain"},
            {"label":"Disaster / Individual Tracking","value":"disaster"},
            {"label":"Reset","value":"reset"},
        ],
        value="reset",
        labelStyle={"display":"inline-block","marginRight":"12px"}
    ),

    html.Br(),

    # --- Data Table (view) ---
    dash_table.DataTable(
        id=datatable_id,
        columns=[],              # will be set by callback
        data=[],                 # will be filled by callback
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        selected_rows=[0],
        style_table={"overflowX":"auto","maxHeight":"400px","overflowY":"scroll"},
        style_cell={"textAlign":"left","minWidth":"120px","width":"120px","maxWidth":"400px"},
        style_header={"fontWeight":"bold"}
    ),

    html.Hr(),

    # --- Charts (views) ---
    html.Div([
        html.Div(id=map_id, className="col s12 m6"),
        dcc.Graph(id=pie_id, className="col s12 m6")
    ], style={"display":"grid","gridTemplateColumns":"1fr 1fr","gap":"16px"})
], style={"padding":"12px"})

#############################################
# Interaction Between Components / Controller
#############################################
def build_query(filter_key: str) -> dict:
    """
    Returns a Mongo query for the requested rescue type.
    filter_key ∈ {"water", "mountain", "disaster", "reset"}
    """
    if filter_key == "reset":
        return {}  # unfiltered

    OR = "$or"
    IN = "$in"

    def breeds_regex(breeds):
        # list of case-insensitive regexes matching each breed token
        return [{ "breed": { "$regex": re.escape(b), "$options": "i" } } for b in breeds]

    if filter_key == "water":
        breeds = ["Labrador Retriever", "Chesapeake Bay Retriever", "Newfoundland"]
        sex = "Intact Female"
        min_wk, max_wk = 26, 156
        return {
            OR: breeds_regex(breeds),
            "sex_upon_outcome": { "$regex": f"^{re.escape(sex)}$", "$options": "i" },
            "age_upon_outcome_in_weeks": { "$gte": min_wk, "$lte": max_wk },
            "animal_type": { "$regex": "^Dog$", "$options": "i" }
        }

    if filter_key == "mountain":
        breeds = ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                  "Siberian Husky", "Rottweiler"]
        sex = "Intact Male"
        min_wk, max_wk = 26, 156
        return {
            OR: breeds_regex(breeds),
            "sex_upon_outcome": { "$regex": f"^{re.escape(sex)}$", "$options": "i" },
            "age_upon_outcome_in_weeks": { "$gte": min_wk, "$lte": max_wk },
            "animal_type": { "$regex": "^Dog$", "$options": "i" }
        }

    if filter_key == "disaster":
        breeds = ["Doberman Pinscher", "German Shepherd", "Golden Retriever",
                  "Bloodhound", "Rottweiler"]
        sex = "Intact Male"
        min_wk, max_wk = 20, 300
        return {
            OR: breeds_regex(breeds),
            "sex_upon_outcome": { "$regex": f"^{re.escape(sex)}$", "$options": "i" },
            "age_upon_outcome_in_weeks": { "$gte": min_wk, "$lte": max_wk },
            "animal_type": { "$regex": "^Dog$", "$options": "i" }
        }

    return {}

def map_children_from_df(df: pd.DataFrame, selected_row: int = 0):
    center = [30.75, -97.48]  # Austin
    if df.empty:
        return [dl.Map(style={'width':'1000px','height':'500px'},
                       center=center, zoom=10,
                       children=[dl.TileLayer(id="base-layer-id")])]

    # row
    row = min(max(selected_row, 0), len(df)-1)

    # column names
    lat_col = next((c for c in df.columns if c.lower() in ("location_lat","latitude","lat","y")), None)
    lon_col = next((c for c in df.columns if c.lower() in ("location_long","longitude","lon","lng","x")), None)
    breed_col = next((c for c in df.columns if c.lower() == "breed"), None)
    name_col  = next((c for c in df.columns if c.lower() == "name"), None)

    lat = float(df.iloc[row][lat_col]) if lat_col and str(df.iloc[row][lat_col]).strip() else center[0]
    lon = float(df.iloc[row][lon_col]) if lon_col and str(df.iloc[row][lon_col]).strip() else center[1]
    breed = str(df.iloc[row][breed_col]) if breed_col else "Unknown"
    name  = str(df.iloc[row][name_col])  if name_col  else "Unknown"

    return [
        dl.Map(style={'width':'1000px','height':'500px'},
               center=center, zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon],
                      children=[dl.Tooltip(breed),
                                dl.Popup([html.H1("Animal Name"),
                                          html.P(name)])])
        ])
    ]

# Filter drives the table (and sets columns)
@app.callback(
    Output(datatable_id, "data"),
    Output(datatable_id, "columns"),
    Input(filter_id, "value")
)
def on_filter_change(filter_value):
    q = build_query(filter_value)
    df = fetch_df(q)
    if "_id" in df.columns:
        df = df.drop(columns=["_id"])
    cols = [{"name": c, "id": c} for c in df.columns]
    return df.to_dict("records"), cols

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output(map_id, "children"),
    Input(datatable_id, "data"),
    Input(datatable_id, "selected_rows")
)
def on_table_select(data, selected_rows):
    df = pd.DataFrame(data)
    sel = selected_rows[0] if selected_rows else 0
    return map_children_from_df(df, sel)

#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output(pie_id, "figure"),
    Input(datatable_id, "data")
)
def on_table_data_change(data):
    df = pd.DataFrame(data)
    if df.empty or "breed" not in df.columns:
        return px.pie(values=[1], names=["No Data"], title="Breed Distribution")
    vc = df["breed"].value_counts().reset_index()
    vc.columns = ["breed","count"]
    fig = px.pie(vc, values="count", names="breed", title="Breed Distribution")
    return fig


# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server(mode="inline", debug=False)


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Dec/2025 17:32:24] "GET /_alive_2117a583-c624-4804-82d2-3d961ac615e7 HTTP/1.1" 200 -


127.0.0.1 - - [09/Dec/2025 17:32:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2025 17:32:24] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2025 17:32:24] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2025 17:32:25] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [09/Dec/2025 17:32:25] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [09/Dec/2025 17:32:25] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [09/Dec/2025 17:32:25] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [09/Dec/2025 17:32:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2025 17:32:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2025 17:32:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2025 17:36:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - -